# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key
geoapify_key

'35c9a212796b40eabde6bf437ecd4679'

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,monrovia,6.3005,-10.7969,26.67,89,9,1.30,LR,1709080541
1,1,port-aux-francais,-49.3500,70.2167,9.03,96,100,10.68,TF,1709080529
2,2,bredasdorp,-34.5322,20.0403,17.22,75,0,6.65,ZA,1709080541
3,3,blackmans bay,-43.0167,147.3167,24.74,50,56,2.37,AU,1709080541
4,4,tsiombe,-25.3000,45.4833,25.30,84,2,0.30,MG,1709080542


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 21
ideal_temp_max = 27
ideal_wind_speed_max = 4.5
ideal_cloudiness = 0
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) &
    (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Wind Speed"] < ideal_wind_speed_max) &
    (city_data_df["Cloudiness"] == ideal_cloudiness)
]
# Drop any rows with null values
city_data_df = city_data_df.dropna()
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,remire-montjoly,4.9167,-52.2667,25.02,83,0,0.00,GF,1709080547
98,98,delta del tigre,-34.7633,-56.3853,22.23,88,0,3.09,UY,1709080563
117,117,puerto san jose,13.9256,-90.8244,25.76,68,0,2.98,GT,1709080568
151,151,vila velha,-20.3297,-40.2925,25.97,94,0,3.09,BR,1709080581
199,199,saint-pierre,-21.3393,55.4781,22.82,78,0,2.57,RE,1709080607
219,219,coronel suarez,-37.4547,-61.9334,21.71,36,0,1.63,AR,1709080626
236,236,realico,-35.0366,-64.2447,21.37,80,0,3.36,AR,1709080631
247,247,kawant,22.1000,74.0500,21.91,43,0,1.69,IN,1709080634
308,308,tanout,14.9709,8.8879,24.37,10,0,3.52,NE,1709080648
327,327,tauranga,-37.6861,176.1667,26.96,56,0,1.34,NZ,1709080652


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df.drop(['Max Temp', 'Wind Speed', 'Cloudiness', 'Date'], axis=1)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
0,0,monrovia,6.3005,-10.7969,89,LR,
1,1,port-aux-francais,-49.3500,70.2167,96,TF,
2,2,bredasdorp,-34.5322,20.0403,75,ZA,
3,3,blackmans bay,-43.0167,147.3167,50,AU,
4,4,tsiombe,-25.3000,45.4833,84,MG,
...,...,...,...,...,...,...,...
578,578,ladario,-19.0047,-57.6017,59,BR,
579,579,mangai,-4.0500,19.5333,96,CD,
580,580,ta`u,-14.2336,-169.5144,74,AS,
581,581,corner brook,48.9667,-57.9484,95,CA,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [2]:
# Define the base URL for the Geoapify API
base_url = "https://api.geoapify.com/v1/places/by-coords-and-type"
# Define the parameters for the Geoapify API request
params = {
    "type": "hotel",
    "radius": 10000,
    "apiKey": "Your-Geoapify-API-Key"
}
# Iterate through each row in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude coordinates of the city
    lat = row['Lat']
    lng = row['Lng']
    # Define the coordinates parameter for the Geoapify API request
    coordinates = f"{lng},{lat}"
    # Add the coordinates parameter to the request parameters
    params['coordinates'] = coordinates
    # Send a GET request to the Geoapify API
    response = requests.get(base_url, params=params)
    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Check if the 'features' key is in the response
        if 'features' in data and len(data['features']) > 0:
            # Get the name of the first hotel
            hotel_name = data['features'][0]['properties']['name']
            # Add the hotel name and country to the hotel_df DataFrame
            hotel_df.at[index, 'Hotel Name'] = hotel_name
            hotel_df.at[index, 'Country'] = row['Country']
        else:
            # If no hotel is found, set the hotel name and country to 'Not found'
            hotel_df.at[index, 'Hotel Name'] = 'Not found'
            hotel_df.at[index, 'Country'] = 'Not found'
    else:
        # If the response status code is not 200, print an error message
        print(f"Error: {response.status_code}")

NameError: name 'hotel_df' is not defined

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [1]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)
# Display the map
map_plot_2

NameError: name 'hotel_df' is not defined